# 要約タスクの評価方法

このノートブックでは、簡単な例を使用して抽象的要約タスクの評価技術について詳しく説明します。[ROUGE](https://aclanthology.org/W04-1013/)や[BERTScore](https://arxiv.org/abs/1904.09675)などの従来の評価手法を探求し、さらにLLMを評価者として使用するより新しいアプローチを紹介します。

要約の品質を評価することは時間のかかるプロセスです。これは一貫性、簡潔性、読みやすさ、内容などの異なる品質指標が関わるためです。`ROUGE`や`BERTScore`などの従来の自動評価指標は具体的で信頼性がありますが、要約の実際の品質とよく相関しない場合があります。これらは人間の判断との相関が比較的低く、特に自由形式の生成タスクにおいてその傾向が見られます（[Liu et al., 2023](https://arxiv.org/pdf/2303.16634.pdf)）。潜在的なバイアスに注意を払いながら、人間による評価、ユーザーフィードバック、またはモデルベースの指標に依存する必要性が高まっています。人間の判断は貴重な洞察を提供しますが、しばしばスケーラブルではなく、コストが高くなる可能性があります。

これらの従来の指標に加えて、抽象的要約を評価するための新しい参照不要の指標として大規模言語モデル（LLM）を活用する手法（[G-Eval](https://arxiv.org/pdf/2303.16634.pdf)）を紹介します。この場合、候補出力をスコア化するために`gpt-4`を使用します。`gpt-4`は流暢で一貫性のあるテキストと低品質のテキストを区別できる言語品質の内部モデルを効果的に学習しています。この内部スコアリングメカニズムを活用することで、LLMによって生成された新しい候補出力の自動評価が可能になります。

## セットアップ

In [ ]:
# Installing necessary packages for the evaluation
# rouge: For evaluating with ROUGE metric
# bert_score: For evaluating with BERTScore
# openai: To interact with OpenAI's API
!pip install rouge --quiet
!pip install bert_score --quiet
!pip install openai --quiet

In [23]:
from openai import OpenAI
import os
import re
import pandas as pd

# Python Implementation of the ROUGE Metric
from rouge import Rouge

# BERTScore leverages the pre-trained contextual embeddings from BERT and matches words in candidate and reference sentences by cosine similarity.
from bert_score import BERTScorer

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))


<IPython.core.display.Javascript object>

## サンプルタスク

このノートブックでは、以下の要約例を使用します。比較のために2つの生成された要約を提供し、`ROUGE`や`BERTScore`などの評価指標で必要となる、人間が書いた参照要約も含まれていることに注意してください。

抜粋（`excerpt`）：

> OpenAIのミッションは、汎用人工知能（AGI）が全人類に利益をもたらすことを確実にすることです。OpenAIは安全で有益なAGIを直接構築しますが、その取り組みが他者がこの成果を達成することを支援する場合も、ミッションが達成されたと考えます。OpenAIはこの目的のためにいくつかの重要な原則に従っています。第一に、広く分散された利益 - AGIの展開に対するあらゆる影響力は、すべての人の利益のために使用され、有害な使用や不当な権力の集中を避けるために用いられます。第二に、長期的な安全性 - OpenAIはAGIを安全にするための研究を行い、そのような研究のAIコミュニティ全体での採用を促進することにコミットしています。第三に、技術的リーダーシップ - OpenAIはAI能力の最前線に立つことを目指しています。第四に、協力的な姿勢 - OpenAIは他の研究・政策機関と積極的に協力し、AGIのグローバルな課題に共に取り組むグローバルコミュニティの創設を目指しています。

要約：

| 参照要約 / `ref_summary`（人間が生成）                                                                                                                                                                                                                                                                                                                                         | 評価要約1 / `eval_summary_1`（システムが生成）                                                                                                                                                                                                                                                                                                                                               | 評価要約2 / `eval_summary_2`（システムが生成）                                                                                                                                                                                                                                                   |
| -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| OpenAIは汎用人工知能（AGI）が全ての人の利益のために使用されることを確実にし、有害な使用や不当な権力の集中を避けることを目指しています。AGIの安全性を研究し、そのような研究をAIコミュニティに促進することにコミットしています。OpenAIはAI能力でのリーダーシップを目指し、AGIの課題に対処するためにグローバルな研究・政策機関と協力しています。 | OpenAIはAGIが全人類に利益をもたらし、有害な使用と権力の集中を避けることを目指しています。安全で有益なAGIの研究を先導し、その採用をグローバルに促進しています。OpenAIはAIにおける技術的リーダーシップを維持しながら、AGIの課題に対処するためにグローバルな機関と協力しています。集団の利益のためにAGIを開発する協力的な世界的取り組みをリードすることを目指しています。 | OpenAIはAGIが全ての人の使用のためにあることを確実にし、有害なものや大きな権力の集中を完全に避けることを目指しています。AGIの安全な側面を研究し、AI関係者の間でこれらの研究を促進することにコミットしています。OpenAIはAIの分野でトップになりたいと考え、AGIの問題を解決するために世界中の研究・政策グループと協力しています。 |

どの要約を個人的に好むか、そしてOpenAIのミッションを本当によく捉えているのはどれかを考えてみてください。

In [9]:
excerpt = "OpenAI's mission is to ensure that artificial general intelligence (AGI) benefits all of humanity. OpenAI will build safe and beneficial AGI directly, but will also consider its mission fulfilled if its work aids others to achieve this outcome. OpenAI follows several key principles for this purpose. First, broadly distributed benefits - any influence over AGI's deployment will be used for the benefit of all, and to avoid harmful uses or undue concentration of power. Second, long-term safety - OpenAI is committed to doing the research to make AGI safe, and to promote the adoption of such research across the AI community. Third, technical leadership - OpenAI aims to be at the forefront of AI capabilities. Fourth, a cooperative orientation - OpenAI actively cooperates with other research and policy institutions, and seeks to create a global community working together to address AGI's global challenges."
ref_summary = "OpenAI aims to ensure artificial general intelligence (AGI) is used for everyone's benefit, avoiding harmful uses or undue power concentration. It is committed to researching AGI safety, promoting such studies among the AI community. OpenAI seeks to lead in AI capabilities and cooperates with global research and policy institutions to address AGI's challenges."
eval_summary_1 = "OpenAI aims to AGI benefits all humanity, avoiding harmful uses and power concentration. It pioneers research into safe and beneficial AGI and promotes adoption globally. OpenAI maintains technical leadership in AI while cooperating with global institutions to address AGI challenges. It seeks to lead a collaborative worldwide effort developing AGI for collective good."
eval_summary_2 = "OpenAI aims to ensure AGI is for everyone's use, totally avoiding harmful stuff or big power concentration. Committed to researching AGI's safe side, promoting these studies in AI folks. OpenAI wants to be top in AI things and works with worldwide research, policy groups to figure AGI's stuff."

<IPython.core.display.Javascript object>

## ROUGEを使用した評価

[ROUGE](https://aclanthology.org/W04-1013/)は、Recall-Oriented Understudy for Gisting Evaluationの略で、主に生成された出力と参照テキスト間の単語の重複を測定します。これは自動要約タスクの評価において広く使用される指標です。その変種の中でも、`ROUGE-L`はシステムが生成した要約と参照要約間の最長連続一致に関する洞察を提供し、システムが元の要約の本質をどの程度保持しているかを測定します。

In [10]:
# function to calculate the Rouge score
def get_rouge_scores(text1, text2):
    rouge = Rouge()
    return rouge.get_scores(text1, text2)


rouge_scores_out = []

# Calculate the ROUGE scores for both summaries using reference
eval_1_rouge = get_rouge_scores(eval_summary_1, ref_summary)
eval_2_rouge = get_rouge_scores(eval_summary_2, ref_summary)

for metric in ["rouge-1", "rouge-2", "rouge-l"]:
    for label in ["F-Score"]:
        eval_1_score = eval_1_rouge[0][metric][label[0].lower()]
        eval_2_score = eval_2_rouge[0][metric][label[0].lower()]

        row = {
            "Metric": f"{metric} ({label})",
            "Summary 1": eval_1_score,
            "Summary 2": eval_2_score,
        }
        rouge_scores_out.append(row)


def highlight_max(s):
    is_max = s == s.max()
    return [
        "background-color: lightgreen" if v else "background-color: white"
        for v in is_max
    ]


rouge_scores_out = (
    pd.DataFrame(rouge_scores_out)
    .set_index("Metric")
    .style.apply(highlight_max, axis=1)
)

rouge_scores_out

,Summary 1,Summary 2
Metric,,
rouge-1 (F-Score),0.488889,0.511628
rouge-2 (F-Score),0.230769,0.163265
rouge-l (F-Score),0.488889,0.511628


<IPython.core.display.Javascript object>

この表は、2つの異なる要約を参照テキストに対して評価した`ROUGE`スコアを示しています。`rouge-1`の場合、要約2が要約1を上回っており、個々の単語のより良い重複を示しています。また`rouge-l`では、要約2がより高いスコアを持ち、最長共通部分列においてより近い一致を意味し、したがって元のテキストの主要な内容と順序を捉える上で、潜在的により良い全体的な要約であることを示しています。要約2は抜粋から直接取り出された多くの単語と短いフレーズを含んでいるため、参照要約との重複がより高くなり、より高い`ROUGE`スコアにつながる可能性があります。

`ROUGE`や[BLEU](https://aclanthology.org/P02-1040.pdf)、[METEOR](https://www.cs.cmu.edu/~alavie/METEOR/)などの類似メトリクスは定量的な測定値を提供しますが、よく生成された要約の真の本質を捉えることにしばしば失敗します。これらは人間のスコアとの相関も悪くなります。流暢で一貫性のある要約を生成することに長けたLLMの進歩を考えると、`ROUGE`のような従来のメトリクスは、これらのモデルを意図せず罰してしまう可能性があります。これは特に、要約が異なって表現されているが、それでも核となる情報を正確に要約している場合に当てはまります。

## BERTScoreを使用した評価

ROUGEは予測テキストと参照テキストの両方における単語の正確な存在に依存しており、根本的な意味論を解釈することができません。ここで[BERTScore](https://arxiv.org/abs/1904.09675)が登場し、BERTモデルからの文脈埋め込みを活用して、機械生成テキストの文脈において予測文と参照文の類似性を評価することを目的としています。両方の文からの埋め込みを比較することで、`BERTScore`は従来のn-gramベースの指標では見逃される可能性のある意味的類似性を捉えます。

In [11]:
# Instantiate the BERTScorer object for English language
scorer = BERTScorer(lang="en")

# Calculate BERTScore for the summary 1 against the excerpt
# P1, R1, F1_1 represent Precision, Recall, and F1 Score respectively
P1, R1, F1_1 = scorer.score([eval_summary_1], [ref_summary])

# Calculate BERTScore for summary 2 against the excerpt
# P2, R2, F2_2 represent Precision, Recall, and F1 Score respectively
P2, R2, F2_2 = scorer.score([eval_summary_2], [ref_summary])

print("Summary 1 F1 Score:", F1_1.tolist()[0])
print("Summary 2 F1 Score:", F2_2.tolist()[0])

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Summary 1 F1 Score: 0.9227314591407776
Summary 2 F1 Score: 0.9189572930335999


<IPython.core.display.Javascript object>

要約間のF1スコアが近い値を示していることは、重要な情報を捉える性能が似ている可能性を示しています。しかし、この小さな差は慎重に解釈する必要があります。`BERTScore`は人間の評価者が理解するような微妙なニュアンスや高次の概念を完全に把握できない可能性があるため、この指標のみに依存すると、要約の実際の品質やニュアンスを誤って解釈する恐れがあります。`BERTScore`と人間の判断、その他の指標を組み合わせた統合的なアプローチにより、より信頼性の高い評価を提供できるでしょう。

## GPT-4を使用した評価

ここでは、生成されたテキストの品質を大規模言語モデルを使用して評価する[G-Eval](https://arxiv.org/pdf/2303.16634.pdf)フレームワークにインスパイアされた、`gpt-4`を使用した**参照不要**テキスト評価器の例を実装します。参照要約との比較に依存する`ROUGE`や`BERTScore`などのメトリクスとは異なり、`gpt-4`ベースの評価器は、正解参照なしに、入力プロンプトとテキストのみに基づいて生成されたコンテンツの品質を評価します。これにより、人間の参照が少ない、または利用できない新しいデータセットやタスクに適用できます。

この手法の概要は以下の通りです：

1. 4つの異なる基準を定義します：
    1. **関連性**: 要約が重要な情報のみを含み、冗長性を排除しているかを評価します。
    2. **一貫性**: 要約の論理的な流れと構成を評価します。
    3. **整合性**: 要約が元文書の事実と一致しているかをチェックします。
    4. **流暢性**: 要約の文法と読みやすさを評価します。
2. これらの各基準に対してプロンプトを作成し、元文書と要約を入力として受け取り、思考の連鎖生成を活用して、各基準に対して1-5の数値スコアを出力するようにモデルを導きます。
3. 定義されたプロンプトで`gpt-4`からスコアを生成し、要約間で比較します。

この実演では、`gpt-4`が各メトリクスに対して離散的なスコア（1-5）を生成する直接スコアリング関数を使用しています。スコアを正規化し、重み付き合計を取ることで、要約の品質と多様性をより良く反映する、より堅牢で連続的なスコアが得られる可能性があります。

In [15]:
# Evaluation prompt template based on G-Eval
EVALUATION_PROMPT_TEMPLATE = """
You will be given one summary written for an article. Your task is to rate the summary on one metric.
Please make sure you read and understand these instructions very carefully. 
Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:

{criteria}

Evaluation Steps:

{steps}

Example:

Source Text:

{document}

Summary:

{summary}

Evaluation Form (scores ONLY):

- {metric_name}
"""

# Metric 1: Relevance

RELEVANCY_SCORE_CRITERIA = """
Relevance(1-5) - selection of important content from the source. \
The summary should include only important information from the source document. \
Annotators were instructed to penalize summaries which contained redundancies and excess information.
"""

RELEVANCY_SCORE_STEPS = """
1. Read the summary and the source document carefully.
2. Compare the summary to the source document and identify the main points of the article.
3. Assess how well the summary covers the main points of the article, and how much irrelevant or redundant information it contains.
4. Assign a relevance score from 1 to 5.
"""

# Metric 2: Coherence

COHERENCE_SCORE_CRITERIA = """
Coherence(1-5) - the collective quality of all sentences. \
We align this dimension with the DUC quality question of structure and coherence \
whereby "the summary should be well-structured and well-organized. \
The summary should not just be a heap of related information, but should build from sentence to a\
coherent body of information about a topic."
"""

COHERENCE_SCORE_STEPS = """
1. Read the article carefully and identify the main topic and key points.
2. Read the summary and compare it to the article. Check if the summary covers the main topic and key points of the article,
and if it presents them in a clear and logical order.
3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria.
"""

# Metric 3: Consistency

CONSISTENCY_SCORE_CRITERIA = """
Consistency(1-5) - the factual alignment between the summary and the summarized source. \
A factually consistent summary contains only statements that are entailed by the source document. \
Annotators were also asked to penalize summaries that contained hallucinated facts.
"""

CONSISTENCY_SCORE_STEPS = """
1. Read the article carefully and identify the main facts and details it presents.
2. Read the summary and compare it to the article. Check if the summary contains any factual errors that are not supported by the article.
3. Assign a score for consistency based on the Evaluation Criteria.
"""

# Metric 4: Fluency

FLUENCY_SCORE_CRITERIA = """
Fluency(1-3): the quality of the summary in terms of grammar, spelling, punctuation, word choice, and sentence structure.
1: Poor. The summary has many errors that make it hard to understand or sound unnatural.
2: Fair. The summary has some errors that affect the clarity or smoothness of the text, but the main points are still comprehensible.
3: Good. The summary has few or no errors and is easy to read and follow.
"""

FLUENCY_SCORE_STEPS = """
Read the summary and evaluate its fluency based on the given criteria. Assign a fluency score from 1 to 3.
"""


def get_geval_score(
    criteria: str, steps: str, document: str, summary: str, metric_name: str
):
    prompt = EVALUATION_PROMPT_TEMPLATE.format(
        criteria=criteria,
        steps=steps,
        metric_name=metric_name,
        document=document,
        summary=summary,
    )
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=5,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response.choices[0].message.content


evaluation_metrics = {
    "Relevance": (RELEVANCY_SCORE_CRITERIA, RELEVANCY_SCORE_STEPS),
    "Coherence": (COHERENCE_SCORE_CRITERIA, COHERENCE_SCORE_STEPS),
    "Consistency": (CONSISTENCY_SCORE_CRITERIA, CONSISTENCY_SCORE_STEPS),
    "Fluency": (FLUENCY_SCORE_CRITERIA, FLUENCY_SCORE_STEPS),
}

summaries = {"Summary 1": eval_summary_1, "Summary 2": eval_summary_2}

data = {"Evaluation Type": [], "Summary Type": [], "Score": []}

for eval_type, (criteria, steps) in evaluation_metrics.items():
    for summ_type, summary in summaries.items():
        data["Evaluation Type"].append(eval_type)
        data["Summary Type"].append(summ_type)
        result = get_geval_score(criteria, steps, excerpt, summary, eval_type)
        score_num = int(result.strip())
        data["Score"].append(score_num)

pivot_df = pd.DataFrame(data, index=None).pivot(
    index="Evaluation Type", columns="Summary Type", values="Score"
)
styled_pivot_df = pivot_df.style.apply(highlight_max, axis=1)
display(styled_pivot_df)

Summary Type,Summary 1,Summary 2
Evaluation Type,,
Coherence,5,3
Consistency,5,5
Fluency,3,2
Relevance,5,4


<IPython.core.display.Javascript object>

全体的に、要約1は4つのカテゴリのうち3つ（一貫性、関連性、流暢性）において要約2を上回る性能を示しています。両方の要約は互いに一致していることが確認されました。この結果は、与えられた評価基準に基づいて、要約1が一般的により好ましいことを示唆している可能性があります。

### 制限事項

LLMベースの指標は、人間が書いたテキストよりもLLMが生成したテキストを好む傾向があることに注意してください。さらに、LLMベースの指標はシステムメッセージ/プロンプトに敏感です。高品質で高コストな評価と自動評価の適切なバランスを取りながら、性能を向上させたり一貫したスコアを得たりするのに役立つ他の技術を実験することをお勧めします。また、この採点方法は現在`gpt-4`のコンテキストウィンドウによって制限されていることも注目に値します。

## 結論

抽象的要約の評価は、さらなる改善のための開かれた領域であり続けています。`ROUGE`、`BLEU`、`BERTScore`などの従来の指標は有用な自動評価を提供しますが、意味的類似性や要約品質の微妙な側面を捉える上で制限があります。さらに、これらは参照出力を必要とし、収集/ラベル付けにコストがかかる場合があります。LLMベースの指標は、一貫性、流暢性、関連性を評価する参照不要の方法として有望です。しかし、これらもLLMによって生成されたテキストを好む潜在的なバイアスを持っています。最終的に、抽象的要約システムを信頼性高く評価するには、自動指標と人間による評価の組み合わせが理想的です。人間による評価は要約品質の包括的な理解を得るために不可欠ですが、効率的で大規模なテストを可能にするために自動評価で補完されるべきです。この分野は、品質、スケーラビリティ、公平性のバランスを取りながら、より堅牢な評価技術を進化させ続けるでしょう。評価方法の進歩は、本番アプリケーションでの進歩を推進するために重要です。

## 参考文献

- [G-EVAL: NLG Evaluation Using GPT-4 with Better Human Alignment](https://arxiv.org/pdf/2303.16634.pdf) - Liu Y, Iter D, Xu Y, Wang S, Xu R, Zhu C. 2023年5月公開。
- [BERTScore: Evaluating Text Generation with BERT](https://arxiv.org/abs/1904.09675) - Zhang T, Kishore V, Wu F, Weinberger KQ, Artzi Y. 2020年2月オンライン公開。
- [ROUGE: A Package for Automatic Evaluation of Summaries](https://aclanthology.org/W04-1013/) - Lin CY. 2004年7月公開。
- [SummEval: Re-evaluating Summarization Evaluation](https://aclanthology.org/2021.tacl-1.24) - Fabbri et al. 2021年4月公開。